# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [4]:
# import libraries
import os
import re
import copy
import time
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler #, OneHotEncoderEstimator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType, ArrayType, FloatType, DoubleType, Row, DateType
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import  MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

%matplotlib inline

In [5]:
# create a Spark session
spark = SparkSession \
     .builder \
     .appName("Sparkify") \
     .getOrCreate()

In [7]:
#check spark installed spark version
print('Version: ', spark.version)

Version:  3.0.2


# Code Prep
In this workspace, all code that is used throughout the notebook is defined.

In [11]:
def load_data(path=''):
    """
    This function loads json data

    Args:
        - path (string): filepath to the data
    Returns:
        - data (json): spark json file
    """
    #create json data from filepath
    data = spark.read.json(path)

    return data

In [248]:
def clean_data(data):
    """
    This function cleans the dataset:
        - delete rows without userIds or sessionIds
        - cast userId to LongType
        - replace special characters of string columns with empty string
        - split ts into useable columns interaction_time, month, date
        - delete ts column
        - split location into seperate columns location, state

    Args:
        data (json): loaded json file
    Returns:
        cleaned_data (json): cleaned json file
    """
    #delete rows without userids or sessionids
    data = data.filter(col('userId').isNotNull() | col('sessionId').isNotNull())

    #cast userId to LongType
    data= data.withColumn("userId", data["userId"].cast(LongType()))

    #replace special characters of strings with empty
    for field in data.schema.fields:
        if field.dataType==StringType():
            data = data.withColumn(field.name, regexp_replace(field.name, '[^a-zA-Z0-9\,\-]', ''))
    
    # split ts into useable columns interaction_time, month, date
    data = data.withColumn('interaction_time', from_unixtime(col('ts').cast(LongType())/1000).cast(TimestampType()))
    data = data.withColumn('month', month(col('interaction_time')))
    data = data.withColumn('date', from_unixtime(col('ts')/1000).cast(DateType()))
    #delete ts column
    data = data.drop('ts')

    #split location into seperate columns location, state
    data = data.withColumn('state', split(col('location'), ',').getItem(1))
    data = data.withColumn('location', split(col('location'),',').getItem(0))

    #create new copy
    cleaned_data = data

    return cleaned_data

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [247]:
#load data
data = load_data(path='mini_sparkify_event_data.json')

In [249]:
#clean data
cleaned_data = clean_data(data)

In [251]:
#display first 5 rows of cleaned data as pandas df
pd.DataFrame(cleaned_data.take(5), columns=cleaned_data.columns)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,registration,sessionId,song,status,userAgent,userId,interaction_time,month,date,state
0,MarthaTilston,LoggedIn,Colin,M,50,Freeman,277.89016,paid,Bakersfield,PUT,...,1538173362000,29,Rockpools,200,Mozilla50WindowsNT61WOW64rv310Gecko20100101Fir...,30,2018-10-01 02:01:57,10,2018-10-01,CA
1,FiveIronFrenzy,LoggedIn,Micah,M,79,Long,236.09424,free,Boston-Cambridge-Newton,PUT,...,1538331630000,8,Canada,200,Mozilla50WindowsNT61WOW64AppleWebKit53736KHTML...,9,2018-10-01 02:03:00,10,2018-10-01,MA-NH
2,AdamLambert,LoggedIn,Colin,M,51,Freeman,282.82730,paid,Bakersfield,PUT,...,1538173362000,29,TimeForMiracles,200,Mozilla50WindowsNT61WOW64rv310Gecko20100101Fir...,30,2018-10-01 02:06:34,10,2018-10-01,CA
3,Enigma,LoggedIn,Micah,M,80,Long,262.71302,free,Boston-Cambridge-Newton,PUT,...,1538331630000,8,KnockingOnForbiddenDoors,200,Mozilla50WindowsNT61WOW64AppleWebKit53736KHTML...,9,2018-10-01 02:06:56,10,2018-10-01,MA-NH
4,DaftPunk,LoggedIn,Colin,M,52,Freeman,223.60771,paid,Bakersfield,PUT,...,1538173362000,29,HarderBetterFasterStronger,200,Mozilla50WindowsNT61WOW64rv310Gecko20100101Fir...,30,2018-10-01 02:11:16,10,2018-10-01,CA


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.